<a href="https://colab.research.google.com/github/cph316/generative_ai/blob/main/%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 📌 STEP 0: 安裝必要套件
!pip install openai
!pip install openai folium gradio
!pip install openai requests
# 🚀下載 stabilityai/stable-diffusion-2-1 模型
# 直接在本地模型生成圖片
!pip install diffusers accelerate transformers safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [3]:
import requests
import openai
import folium
import urllib.parse
from geopy.geocoders import Nominatim
from IPython.display import display, HTML
import gradio as gr
from openai import OpenAI
from google.colab import userdata
import base64
import time

# ✅ 模型切換區
provider = "groq"  # 可改為 "openai"
api_keys = {
    "openai": userdata.get("OpenAI"),
    "groq": userdata.get("Groq"),
}
models = {
    "openai": "gpt-4",
    "groq": "llama3-8b-8192",
}
api_key = api_keys[provider]
model = models[provider]
print(f"✅ 使用模型：{provider.upper()}（{model}）")

# ✅ 回答問題
def llm_reply(prompt):
    try:
        if provider == "groq":
            url = "https://api.groq.com/openai/v1/chat/completions"
            headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
            body = {
                "model": model,
                "messages": [
                    {"role": "system", "content": "你是一個旅遊助手，請用不超過 50 字繁體中文簡短回答。"},
                    {"role": "user", "content": prompt}
                ],
                "temperature": 0.7,
                "max_tokens": 100
            }
            response = requests.post(url, headers=headers, json=body)
            return response.json()["choices"][0]["message"]["content"] if response.status_code == 200 \
                else f"❌ Groq 錯誤：{response.status_code} - {response.text}"

        elif provider == "openai":
            client = OpenAI(api_key=api_key)
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "你是一個旅遊助手，請用不超過 50 字繁體中文簡短回答。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=100
            )
            return response.choices[0].message.content

        else:
            return "❌ 不支援的 provider"
    except Exception as e:
        return f"❌ 發生例外錯誤：{e}"

# ✅ 地點抽取
def extract_location_by_llm(prompt):
    system_instruction = "你是一個 NLP 模組，只負責從句子中抽出地點名稱，不解釋、不翻譯、不多話。只回傳最有可能的地名，例如：'東京塔'"
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {"Authorization": f"Bearer {api_keys['groq']}", "Content-Type": "application/json"}
    body = {
        "model": "llama3-8b-8192",
        "messages": [
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0,
        "max_tokens": 30
    }
    response = requests.post(url, headers=headers, json=body)
    return response.json()["choices"][0]["message"]["content"].strip() if response.status_code == 200 else ""

# ✅ 顯示地圖
def generate_map_html(location_name):
    geolocator = Nominatim(user_agent="travel_assistant")
    location = geolocator.geocode(location_name)
    if location:
        m = folium.Map(location=[location.latitude, location.longitude], zoom_start=13)
        folium.Marker([location.latitude, location.longitude], popup=f"{location_name}").add_to(m)
        return m._repr_html_()
    else:
        return "⚠️ 找不到地點，請確認輸入是否正確"

# ✅ 產生圖片封面（用 OpenAI DALL·E，這裡先簡化回傳字串）

HF_TOKEN = userdata.get("HF_TOKEN")
HF_SD_MODEL = "stabilityai/stable-diffusion-2"

def generate_cover_image(prompt):
    try:
        encoded_text = urllib.parse.quote(prompt)
        image_url = f"https://dummyimage.com/512x512/cccccc/000000.png&text={encoded_text}"
        return image_url
    except Exception as e:
        return None

# def generate_cover_image(prompt):
#     try:
#         print("⏳ 開始向 Hugging Face 請求圖片生成，請耐心等待...")
#         start_time = time.time()

#         url = f"https://api-inference.huggingface.co/models/{HF_SD_MODEL}"
#         headers = {
#             "Authorization": f"Bearer {HF_TOKEN}",
#             "Content-Type": "application/json"
#         }
#         payload = {"inputs": f"travel guide cover, {prompt}"}

#         response = requests.post(url, headers=headers, json=payload, timeout=120)  # 設定最大等待 120 秒

#         duration = time.time() - start_time
#         print(f"✅ 請求完成，用時 {duration:.1f} 秒")

#         if response.status_code == 200:
#             image_bytes = response.content
#             image_base64 = base64.b64encode(image_bytes).decode("utf-8")
#             return f"data:image/png;base64,{image_base64}"
#         else:
#             print(f"⚠️ HF API 錯誤：{response.status_code} - {response.text}")
#             return None
#     except requests.exceptions.Timeout:
#         print("⚠️ 請求超時，請稍後再試。")
#         return None
#     except Exception as e:
#         print(f"⚠️ 發生例外錯誤：{e}")
#         return None

# ✅ 整合 UI 邏輯
def travel_assistant(prompt, options):
    reply = llm_reply(prompt)
    map_html = ""
    image_url = None

    if "Map" in options:
        location = extract_location_by_llm(prompt)
        print("🔍 抽取地點：", location)
        map_html = generate_map_html(location)

    if "Image" in options:
        # 先放一個「生成中」的占位圖
        image_url = "https://dummyimage.com/512x512/cccccc/000000.png&text=生成中"

        # 再試著生成真正的圖片
        real_image = generate_cover_image(prompt)
        if real_image:
            image_url = real_image
        else:
            image_url = "https://dummyimage.com/512x512/ff0000/ffffff.png&text=生成失敗"


    return reply, map_html, image_url

# ✅ Gradio UI 介面
with gr.Blocks() as demo:
    gr.Markdown("## 🧳 AI 旅遊小助手 · 專屬手冊製作工具")

    with gr.Row():
        prompt = gr.Textbox(label="請輸入你的旅遊需求", lines=3)
        options = gr.CheckboxGroup(choices=["Map", "Image"], label="選擇功能：")

    submit = gr.Button("送出")
    reply = gr.Textbox(label="AI 旅遊建議")
    map_display = gr.HTML()
    image_display = gr.Image(label="旅遊封面圖")

    submit.click(
        fn=travel_assistant,
        inputs=[prompt, options],
        outputs=[reply, map_display, image_display]
    )

demo.launch()

✅ 使用模型：GROQ（llama3-8b-8192）
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc8e080a2a30fa0ee2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
